In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import os

# List of states and their corresponding file paths
states = ['ce', 'go', 'mg', 'pr']
climate_files = [f'aggregated_climate_data_{state}.csv' for state in states]
dengue_files = [f'aggregated_dengue_data_{state}.csv' for state in states]

# Function to preprocess data, train model, and return MAPE and results DataFrame
def train_and_evaluate(state, climate_file, dengue_file, train_col, target_col, epochs, batch_size):
    # Load data
    dengue_data = pd.read_csv(dengue_file)
    climate_data = pd.read_csv(climate_file)

    # Filter for train and target columns
    dengue_data[train_col] = dengue_data[train_col].astype(bool)
    dengue_data[target_col] = dengue_data[target_col].astype(bool)

    dengue_train = dengue_data[dengue_data[train_col]]
    dengue_target = dengue_data[dengue_data[target_col]]

    # Convert date columns to datetime
    dengue_train['date'] = pd.to_datetime(dengue_train['date'])
    dengue_target['date'] = pd.to_datetime(dengue_target['date'])
    climate_data['date'] = pd.to_datetime(climate_data['date'])

    # Merge the datasets on 'date'
    train_data = pd.merge(dengue_train, climate_data, on='date')
    target_data = pd.merge(dengue_target, climate_data, on='date')

    # Select relevant columns excluding 'epiweek'
    columns_to_keep = ['date', 'casos', 'temp_min', 'temp_med', 'temp_max',
                       'precip_min', 'precip_med', 'precip_max', 'precip_tot',
                       'pressure_min', 'pressure_med', 'pressure_max',
                       'rel_humid_min', 'rel_humid_med', 'rel_humid_max',
                       'thermal_range', 'rainy_days']

    train_data = train_data[columns_to_keep]
    target_data = target_data[columns_to_keep]

    # Apply Variance Threshold
    climate_features = train_data.drop(columns=['casos', 'date'])
    selector = VarianceThreshold(threshold=0.1)
    selected_features = selector.fit_transform(climate_features)

    # Get the selected feature names
    selected_feature_names = climate_features.columns[selector.get_support()]
    selected_feature_names = list(selected_feature_names)
    print(f"{state.upper()} - Selected Features:", selected_feature_names)

    # Add 'casos' and 'date' to the selected features
    selected_feature_names = ['casos', 'date'] + selected_feature_names

    # Update train_data and target_data with selected features
    train_data = train_data[selected_feature_names]
    target_data = target_data[selected_feature_names]

    # Separate the target variable and climate features excluding 'date'
    X_train_climate = train_data.drop(columns=['casos', 'date'])
    X_target_climate = target_data.drop(columns=['casos', 'date'])

    # Standardize the climate features
    scaler_climate = StandardScaler()
    X_train_climate_scaled = scaler_climate.fit_transform(X_train_climate)
    X_target_climate_scaled = scaler_climate.transform(X_target_climate)

    # Scale the 'casos' column separately
    scaler_casos = MinMaxScaler()
    train_data['casos_scaled'] = scaler_casos.fit_transform(train_data[['casos']])
    target_data['casos_scaled'] = scaler_casos.transform(target_data[['casos']])

    # Apply PCA to climate features
    pca = PCA(n_components=0.95)
    X_train_pca = pca.fit_transform(X_train_climate_scaled)
    X_target_pca = pca.transform(X_target_climate_scaled)

    # Create DataFrames with PCA components
    n_components = X_train_pca.shape[1]
    train_pca_df = pd.DataFrame(X_train_pca, index=train_data.index, columns=[f'PC{i+1}' for i in range(n_components)])
    target_pca_df = pd.DataFrame(X_target_pca, index=target_data.index, columns=[f'PC{i+1}' for i in range(n_components)])

    # Add the scaled target variable 'casos_scaled' and 'date' to the PCA DataFrames
    train_pca_df['casos_scaled'] = train_data['casos_scaled']
    train_pca_df['date'] = train_data['date']
    target_pca_df['casos_scaled'] = target_data['casos_scaled']
    target_pca_df['date'] = target_data['date']

    # Set date as index
    train_pca_df.set_index('date', inplace=True)
    target_pca_df.set_index('date', inplace=True)

    # Prepare the data for LSTM
    X_train = train_pca_df.drop(columns=['casos_scaled']).values
    y_train = train_pca_df['casos_scaled'].values
    X_target = target_pca_df.drop(columns=['casos_scaled']).values

    # Define a more complex LSTM model
    model = Sequential()
    model.add(LSTM(1000, activation='relu', return_sequences=True, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(0.3))
    model.add(LSTM(500, activation='relu', return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(250, activation='relu', return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(100, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # Reshape the data for LSTM
    X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_target_reshaped = X_target.reshape((X_target.shape[0], 1, X_target.shape[1]))

    # Train the model
    model.fit(X_train_reshaped, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

    # Save the model
    model.save(f'model_{state}_{train_col}.h5')


    # Make predictions
    y_pred = model.predict(X_target_reshaped)

    # Inverse transform the predictions
    y_pred_inv = scaler_casos.inverse_transform(y_pred.reshape(-1, 1)).flatten()

    # Set values less than 0 to 0
    y_pred_inv[y_pred_inv < 0] = 0
    # Convert the predictions to integers
    y_pred_inv = y_pred_inv.astype(int)

    # Inverse transform the actual values
    y_target_inv = scaler_casos.inverse_transform(target_data['casos_scaled'].values.reshape(-1, 1)).flatten()

    # Calculate MAE and MAPE
    mae = mean_absolute_error(y_target_inv, y_pred_inv)
    mape = mean_absolute_percentage_error(y_target_inv, y_pred_inv) * 100

    # Create a DataFrame with the date, real cases, and predicted cases
    results_df = pd.DataFrame({
        'date': target_pca_df.index,
        'real': y_target_inv,
        'predict': y_pred_inv
    })

    return mape, results_df

# Define the grid search parameters
epochs_list = [50, 100, 200]
batch_size_list = [16, 32]

# Loop through each state and each task, and perform grid search for the best LSTM parameters
for state, climate_file, dengue_file in zip(states, climate_files, dengue_files):
    best_mape_1 = float('inf')
    best_results_df_1 = None
    best_params_1 = None

    best_mape_2 = float('inf')
    best_results_df_2 = None
    best_params_2 = None

    # First task: Filter for train_1 == True and target_1 ==target_1 == True
    for epochs in epochs_list:
        for batch_size in batch_size_list:
            mape1, results_df1 = train_and_evaluate(state, climate_file, dengue_file, 'train_1', 'target_1', epochs, batch_size)
            if mape1 < best_mape_1:
                best_mape_1 = mape1
                best_results_df_1 = results_df1
                best_params_1 = (epochs, batch_size)

    best_results_df_1.to_csv(f'{state}_1.csv', index=False)
    print(f"Results saved for {state} task 1 with params: epochs={best_params_1[0]}, batch_size={best_params_1[1]}")

    # Second task: Filter for train_2 == True and target_2 == True
    for epochs in epochs_list:
        for batch_size in batch_size_list:
            mape2, results_df2 = train_and_evaluate(state, climate_file, dengue_file, 'train_2', 'target_2', epochs, batch_size)
            if mape2 < best_mape_2:
                best_mape_2 = mape2
                best_results_df_2 = results_df2
                best_params_2 = (epochs, batch_size)

    best_results_df_2.to_csv(f'{state}_2.csv', index=False)
    print(f"Results saved for {state} task 2 with params: epochs={best_params_2[0]}, batch_size={best_params_2[1]}")

print("Model training and evaluation completed for all states and tasks.")

<ipython-input-1-c0d697b98a8b>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengue_train['date'] = pd.to_datetime(dengue_train['date'])
<ipython-input-1-c0d697b98a8b>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengue_target['date'] = pd.to_datetime(dengue_target['date'])


CE - Selected Features: ['temp_min', 'temp_med', 'temp_max', 'precip_max', 'precip_tot', 'rel_humid_min', 'rel_humid_med', 'rel_humid_max', 'thermal_range', 'rainy_days']


Epoch 1/50
42/42 [==============================] - 9s 14ms/step - loss: 0.0301
Epoch 2/50
42/42 [==============================] - 1s 14ms/step - loss: 0.0236
Epoch 3/50
42/42 [==============================] - 1s 13ms/step - loss: 0.0232
Epoch 4/50
42/42 [==============================] - 1s 14ms/step - loss: 0.0220
Epoch 5/50
42/42 [==============================] - 1s 14ms/step - loss: 0.0219
Epoch 6/50
42/42 [==============================] - 1s 13ms/step - loss: 0.0216
Epoch 7/50
42/42 [==============================] - 1s 14ms/step - loss: 0.0213
Epoch 8/50
42/42 [==============================] - 1s 13ms/step - loss: 0.0212
Epoch 9/50
42/42 [==============================] - 1s 22ms/step - loss: 0.0217
Epoch 10/50
42/42 [==============================] - 1s 24ms/step - loss: 0.0208
Epoch 11/50
42/42 [==============================] - 1s 24ms/step - loss: 0.0203
Epoch 12/50
42/42 [==============================] - 1s 15ms/step - loss: 0.0205
Epoch 13/50
42/42 [==================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2/2 [==============================] - 0s 7ms/step


...

2/2 [==============================] - 0s 8ms/step


<ipython-input-1-c0d697b98a8b>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengue_train['date'] = pd.to_datetime(dengue_train['date'])
<ipython-input-1-c0d697b98a8b>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengue_target['date'] = pd.to_datetime(dengue_target['date'])


GO - Selected Features: ['temp_min', 'temp_med', 'temp_max', 'precip_max', 'precip_tot', 'rel_humid_min', 'rel_humid_med', 'rel_humid_max', 'thermal_range', 'rainy_days']


Epoch 1/200
42/42 [==============================] - 6s 14ms/step - loss: 0.0523
Epoch 2/200
42/42 [==============================] - 1s 14ms/step - loss: 0.0379
Epoch 3/200
42/42 [==============================] - 1s 15ms/step - loss: 0.0356
Epoch 4/200
42/42 [==============================] - 1s 14ms/step - loss: 0.0362
Epoch 5/200
42/42 [==============================] - 1s 15ms/step - loss: 0.0339
Epoch 6/200
42/42 [==============================] - 1s 15ms/step - loss: 0.0343
Epoch 7/200
42/42 [==============================] - 1s 15ms/step - loss: 0.0342
Epoch 8/200
42/42 [==============================] - 1s 15ms/step - loss: 0.0331
Epoch 9/200
42/42 [==============================] - 1s 14ms/step - loss: 0.0332
Epoch 10/200
42/42 [==============================] - 1s 15ms/step - loss: 0.0345
Epoch 11/200
42/42 [==============================] - 1s 14ms/step - loss: 0.0331
Epoch 12/200
42/42 [==============================] - 1s 15ms/step - loss: 0.0333
Epoch 13/200
42/42 [=====

KeyboardInterrupt: 

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error

# List of states and their corresponding file paths
#states = ['ce', 'go', 'mg', 'pr']
#climate_files = [f'aggregated_climate_data_{state}.csv' for state in states]
#dengue_files = [f'aggregated_dengue_data_{state}.csv' for state in states]

# Load data
dengue_file_path = 'aggregated_dengue_data_ce.csv'
climate_file_path = 'aggregated_climate_data_ce.csv'
dengue_data = pd.read_csv(dengue_file_path)
climate_data = pd.read_csv(climate_file_path)

# Filter for train_1 == True and target_1 == True
dengue_data['train_1'] = dengue_data['train_1'].astype(bool)
dengue_data['target_1'] = dengue_data['target_1'].astype(bool)

dengue_train = dengue_data[dengue_data['train_1']]
dengue_target = dengue_data[dengue_data['target_1']]

# Convert date columns to datetime
dengue_train['date'] = pd.to_datetime(dengue_train['date'])
dengue_target['date'] = pd.to_datetime(dengue_target['date'])
climate_data['date'] = pd.to_datetime(climate_data['date'])

# Merge the datasets on 'date'
train_data = pd.merge(dengue_train, climate_data, on='date')
target_data = pd.merge(dengue_target, climate_data, on='date')

# Select relevant columns excluding 'epiweek'
columns_to_keep = ['date', 'casos', 'temp_min', 'temp_med', 'temp_max',
                   'precip_min', 'precip_med', 'precip_max', 'precip_tot',
                   'pressure_min', 'pressure_med', 'pressure_max',
                   'rel_humid_min', 'rel_humid_med', 'rel_humid_max',
                   'thermal_range', 'rainy_days']

train_data = train_data[columns_to_keep]
target_data = target_data[columns_to_keep]

# Apply Variance Threshold
climate_features = train_data.drop(columns=['casos', 'date'])
selector = VarianceThreshold(threshold=0.1)
selected_features = selector.fit_transform(climate_features)

# Get the selected feature names
selected_feature_names = climate_features.columns[selector.get_support()]
selected_feature_names = list(selected_feature_names)
print("Selected Features:", selected_feature_names)

# Add 'casos' and 'date' to the selected features
selected_feature_names = ['casos', 'date'] + selected_feature_names

# Update train_data and target_data with selected features
train_data = train_data[selected_feature_names]
target_data = target_data[selected_feature_names]

# Separate the target variable and climate features excluding 'date'
X_train_climate = train_data.drop(columns=['casos', 'date'])
X_target_climate = target_data.drop(columns=['casos', 'date'])

# Standardize the climate features
scaler_climate = StandardScaler()
X_train_climate_scaled = scaler_climate.fit_transform(X_train_climate)
X_target_climate_scaled = scaler_climate.transform(X_target_climate)

# Scale the 'casos' column separately
scaler_casos = MinMaxScaler()
train_data['casos_scaled'] = scaler_casos.fit_transform(train_data[['casos']])
target_data['casos_scaled'] = scaler_casos.transform(target_data[['casos']])

# Apply PCA to climate features
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_climate_scaled)
X_target_pca = pca.transform(X_target_climate_scaled)

# Create DataFrames with PCA components
n_components = X_train_pca.shape[1]
train_pca_df = pd.DataFrame(X_train_pca, index=train_data.index, columns=[f'PC{i+1}' for i in range(n_components)])
target_pca_df = pd.DataFrame(X_target_pca, index=target_data.index, columns=[f'PC{i+1}' for i in range(n_components)])

# Add the scaled target variable 'casos_scaled' and 'date' to the PCA DataFrames
train_pca_df['casos_scaled'] = train_data['casos_scaled']
train_pca_df['date'] = train_data['date']
target_pca_df['casos_scaled'] = target_data['casos_scaled']
target_pca_df['date'] = target_data['date']

# Set date as index
train_pca_df.set_index('date', inplace=True)
target_pca_df.set_index('date', inplace=True)

# Prepare the data for LSTM
X_train = train_pca_df.drop(columns=['casos_scaled']).values
y_train = train_pca_df['casos_scaled'].values
X_target = target_pca_df.drop(columns=['casos_scaled']).values


# Define a more complex LSTM model
model = Sequential()
model.add(LSTM(1000, activation='relu', return_sequences=True, input_shape=(1, X_train.shape[1])))
model.add(Dropout(0.3))
model.add(LSTM(500, activation='relu', return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(250, activation='relu', return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(100, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Reshape the data for LSTM
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_target_reshaped = X_target.reshape((X_target.shape[0], 1, X_target.shape[1]))

# Train the model
#model.fit(X_train_reshaped, y_train, epochs=50, batch_size=16, verbose=1)
model.fit(X_train_reshaped, y_train, epochs=500, batch_size=32,verbose=1)

# Save the model
model.save('model_ce_task1.h5')


# Make predictions
y_pred = model.predict(X_target_reshaped)

# Inverse transform the predictions
y_pred_inv = scaler_casos.inverse_transform(y_pred.reshape(-1, 1)).flatten()

# Set values less than 0 to 0
y_pred_inv[y_pred_inv < 0] = 0
# Convert the predictions to integers
y_pred_inv = y_pred_inv.astype(int)

# Inverse transform the actual values
y_target_inv = scaler_casos.inverse_transform(target_data['casos_scaled'].values.reshape(-1, 1)).flatten()


# Print or return the real cases
print("Real Cases (Actual):", y_target_inv)
print("Real Cases (Predicted):", y_pred_inv)

# Calculate MAE and MAPE
mae = mean_absolute_error(y_target_inv, y_pred_inv)
mape = np.mean(np.abs((y_target_inv - y_pred_inv) / y_target_inv)) * 100

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")

<ipython-input-4-e42ada829560>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengue_train['date'] = pd.to_datetime(dengue_train['date'])
<ipython-input-4-e42ada829560>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengue_target['date'] = pd.to_datetime(dengue_target['date'])


Selected Features: ['temp_min', 'temp_med', 'temp_max', 'precip_max', 'precip_tot', 'rel_humid_min', 'rel_humid_med', 'rel_humid_max', 'thermal_range', 'rainy_days']


Epoch 1/500
21/21 [==============================] - 6s 16ms/step - loss: 0.0344
Epoch 2/500
21/21 [==============================] - 0s 14ms/step - loss: 0.0259
Epoch 3/500
21/21 [==============================] - 0s 15ms/step - loss: 0.0242
Epoch 4/500
21/21 [==============================] - 0s 15ms/step - loss: 0.0226
Epoch 5/500
21/21 [==============================] - 0s 15ms/step - loss: 0.0220
Epoch 6/500
21/21 [==============================] - 0s 17ms/step - loss: 0.0227
Epoch 7/500
21/21 [==============================] - 1s 27ms/step - loss: 0.0215
Epoch 8/500
21/21 [==============================] - 1s 26ms/step - loss: 0.0211
Epoch 9/500
21/21 [==============================] - 1s 29ms/step - loss: 0.0212
Epoch 10/500
21/21 [==============================] - 1s 26ms/step - loss: 0.0211
Epoch 11/500
21/21 [==============================] - 0s 19ms/step - loss: 0.0209
Epoch 12/500
21/21 [==============================] - 0s 14ms/step - loss: 0.0207
Epoch 13/500
21/21 [=====

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2/2 [==============================] - 0s 6ms/step
Real Cases (Actual): [216. 186. 132. 133. 165. 161. 143. 142. 106.  74.  47.  65. 163. 127.
 170. 226. 240. 246. 274. 354. 482. 577. 561. 614. 581. 444. 543. 509.
 476. 636. 689. 666. 615. 512. 475. 514. 424. 362. 340. 339. 291. 253.
 272. 216. 210. 224. 188. 134. 133. 104.  88.]
Real Cases (Predicted): [ 418  418  295 1683 1086 4737 2694 1021  396  382 4602 5022 1437  480
 1152 1801 1848 4790 1988  900 2240 1363  549 4098 3376 3845 1966 4984
  953 2715 2671  812 1921 1822 2132  785 1015 1556 1575  328  620  619
  378  780  657  324  300  418  405  392  418]
Mean Absolute Error (MAE): 1321.0
Mean Absolute Percentage Error (MAPE): 760.651178804038%


In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error


# Load data
dengue_file_path = 'aggregated_dengue_data_am.csv'
climate_file_path = 'aggregated_climate_data_am.csv'
dengue_data = pd.read_csv(dengue_file_path)
climate_data = pd.read_csv(climate_file_path)

# Filter for train_1 == True and target_1 == True
dengue_data['train_1'] = dengue_data['train_1'].astype(bool)
dengue_data['target_1'] = dengue_data['target_1'].astype(bool)

dengue_train = dengue_data[dengue_data['train_1']]
dengue_target = dengue_data[dengue_data['target_1']]

# Convert date columns to datetime
dengue_train['date'] = pd.to_datetime(dengue_train['date'])
dengue_target['date'] = pd.to_datetime(dengue_target['date'])
climate_data['date'] = pd.to_datetime(climate_data['date'])

# Merge the datasets on 'date'
train_data = pd.merge(dengue_train, climate_data, on='date')
target_data = pd.merge(dengue_target, climate_data, on='date')

# Select relevant columns excluding 'epiweek'
columns_to_keep = ['date', 'casos', 'temp_min', 'temp_med', 'temp_max',
                   'precip_min', 'precip_med', 'precip_max', 'precip_tot',
                   'pressure_min', 'pressure_med', 'pressure_max',
                   'rel_humid_min', 'rel_humid_med', 'rel_humid_max',
                   'thermal_range', 'rainy_days']

train_data = train_data[columns_to_keep]
target_data = target_data[columns_to_keep]

# Apply Variance Threshold
climate_features = train_data.drop(columns=['casos', 'date'])
selector = VarianceThreshold(threshold=0.1)
selected_features = selector.fit_transform(climate_features)

# Get the selected feature names
selected_feature_names = climate_features.columns[selector.get_support()]
selected_feature_names = list(selected_feature_names)
print("Selected Features:", selected_feature_names)

# Add 'casos' and 'date' to the selected features
selected_feature_names = ['casos', 'date'] + selected_feature_names

# Update train_data and target_data with selected features
train_data = train_data[selected_feature_names]
target_data = target_data[selected_feature_names]

# Separate the target variable and climate features excluding 'date'
X_train_climate = train_data.drop(columns=['casos', 'date'])
X_target_climate = target_data.drop(columns=['casos', 'date'])

# Standardize the climate features
scaler_climate = StandardScaler()
X_train_climate_scaled = scaler_climate.fit_transform(X_train_climate)
X_target_climate_scaled = scaler_climate.transform(X_target_climate)

# Scale the 'casos' column separately
scaler_casos = MinMaxScaler()
train_data['casos_scaled'] = scaler_casos.fit_transform(train_data[['casos']])
target_data['casos_scaled'] = scaler_casos.transform(target_data[['casos']])

# Apply PCA to climate features
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_climate_scaled)
X_target_pca = pca.transform(X_target_climate_scaled)

# Create DataFrames with PCA components
n_components = X_train_pca.shape[1]
train_pca_df = pd.DataFrame(X_train_pca, index=train_data.index, columns=[f'PC{i+1}' for i in range(n_components)])
target_pca_df = pd.DataFrame(X_target_pca, index=target_data.index, columns=[f'PC{i+1}' for i in range(n_components)])

# Add the scaled target variable 'casos_scaled' and 'date' to the PCA DataFrames
train_pca_df['casos_scaled'] = train_data['casos_scaled']
train_pca_df['date'] = train_data['date']
target_pca_df['casos_scaled'] = target_data['casos_scaled']
target_pca_df['date'] = target_data['date']

# Set date as index
train_pca_df.set_index('date', inplace=True)
target_pca_df.set_index('date', inplace=True)

# Prepare the data for LSTM
X_train = train_pca_df.drop(columns=['casos_scaled']).values
y_train = train_pca_df['casos_scaled'].values
X_target = target_pca_df.drop(columns=['casos_scaled']).values


# Define a more complex LSTM model
model = Sequential()
model.add(LSTM(1000, activation='relu', return_sequences=True, input_shape=(1, X_train.shape[1])))
model.add(Dropout(0.3))
model.add(LSTM(500, activation='relu', return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(250, activation='relu', return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(100, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Reshape the data for LSTM
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_target_reshaped = X_target.reshape((X_target.shape[0], 1, X_target.shape[1]))

# Train the model
#model.fit(X_train_reshaped, y_train, epochs=50, batch_size=16, verbose=1)
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=16,verbose=1)

# Save the model
model.save('model_am_task1.h5')


# Make predictions
y_pred = model.predict(X_target_reshaped)

# Inverse transform the predictions
y_pred_inv = scaler_casos.inverse_transform(y_pred.reshape(-1, 1)).flatten()

# Set values less than 0 to 0
y_pred_inv[y_pred_inv < 0] = 0
# Convert the predictions to integers
y_pred_inv = y_pred_inv.astype(int)

# Inverse transform the actual values
y_target_inv = scaler_casos.inverse_transform(target_data['casos_scaled'].values.reshape(-1, 1)).flatten()


# Print or return the real cases
print("Real Cases (Actual):", y_target_inv)
print("Real Cases (Predicted):", y_pred_inv)

# Calculate MAE and MAPE
mae = mean_absolute_error(y_target_inv, y_pred_inv)
mape = np.mean(np.abs((y_target_inv - y_pred_inv) / y_target_inv)) * 100

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")

# Create a DataFrame with the date, real cases, and predicted cases
results_df = pd.DataFrame({
    'date': target_pca_df.index,
    'real': y_target_inv,
    'predict': y_pred_inv
})

# Save the DataFrame to a CSV file
results_df.to_csv('am(1).csv', index=False)

print("Results saved")

<ipython-input-6-9936cf8d64ed>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengue_train['date'] = pd.to_datetime(dengue_train['date'])
<ipython-input-6-9936cf8d64ed>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengue_target['date'] = pd.to_datetime(dengue_target['date'])


Selected Features: ['temp_min', 'temp_med', 'temp_max', 'precip_max', 'precip_tot', 'rel_humid_min', 'rel_humid_med', 'rel_humid_max', 'thermal_range', 'rainy_days']


Epoch 1/100
42/42 [==============================] - 7s 14ms/step - loss: 0.0077
Epoch 2/100
42/42 [==============================] - 1s 14ms/step - loss: 0.0074
Epoch 3/100
42/42 [==============================] - 1s 19ms/step - loss: 0.0074
Epoch 4/100
42/42 [==============================] - 1s 14ms/step - loss: 0.0071
Epoch 5/100
42/42 [==============================] - 1s 14ms/step - loss: 0.0071
Epoch 6/100
42/42 [==============================] - 1s 14ms/step - loss: 0.0069
Epoch 7/100
42/42 [==============================] - 1s 14ms/step - loss: 0.0071
Epoch 8/100
42/42 [==============================] - 1s 14ms/step - loss: 0.0069
Epoch 9/100
42/42 [==============================] - 1s 14ms/step - loss: 0.0068
Epoch 10/100
42/42 [==============================] - 1s 14ms/step - loss: 0.0068
Epoch 11/100
42/42 [==============================] - 1s 15ms/step - loss: 0.0068
Epoch 12/100
42/42 [==============================] - 1s 24ms/step - loss: 0.0067
Epoch 13/100
42/42 [=====

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2/2 [==============================] - 0s 8ms/step
Real Cases (Actual): [250. 196. 161. 147. 129. 117. 118. 110.  87.  52.  44.  81. 198. 195.
 198. 214. 251. 277. 180. 148. 167. 143. 125. 119. 122. 125. 106. 153.
 135. 143. 127. 131.  89.  65.  75.  69.  78.  90. 103. 115.  93.  89.
  57.  46.  75.  65.  67.  77.  75.  73.  57.]
Real Cases (Predicted): [172 359 218 149 165 202 207 255 210 203 358 225 245 413 285 260 699 294
 303 259 361 257 434 179 311 353 245 249 264 304 200 269 252 187 218 198
 179 167 170 171 168 177 169 163 162 142 159 128 154 161 148]
Mean Absolute Error (MAE): 120.17647058823529
Mean Absolute Percentage Error (MAPE): 122.31070528920984%
Results saved


In [7]:
!pip install prophet

In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from prophet import Prophet

# Load data
dengue_file_path = 'aggregated_dengue_data_ce.csv'
climate_file_path = 'aggregated_climate_data_ce.csv'
dengue_data = pd.read_csv(dengue_file_path)
climate_data = pd.read_csv(climate_file_path)

# Filter for train_1 == True and target_1 == True
dengue_data['train_1'] = dengue_data['train_1'].astype(bool)
dengue_data['target_1'] = dengue_data['target_1'].astype(bool)

dengue_train = dengue_data[dengue_data['train_1']]
dengue_target = dengue_data[dengue_data['target_1']]

# Convert date columns to datetime
dengue_train['date'] = pd.to_datetime(dengue_train['date'])
dengue_target['date'] = pd.to_datetime(dengue_target['date'])
climate_data['date'] = pd.to_datetime(climate_data['date'])

# Merge the datasets on 'date'
train_data = pd.merge(dengue_train, climate_data, on='date')
target_data = pd.merge(dengue_target, climate_data, on='date')

# Select relevant columns excluding 'epiweek'
columns_to_keep = ['date', 'casos', 'temp_min', 'temp_med', 'temp_max',
                   'precip_min', 'precip_med', 'precip_max', 'precip_tot',
                   'pressure_min', 'pressure_med', 'pressure_max',
                   'rel_humid_min', 'rel_humid_med', 'rel_humid_max',
                   'thermal_range', 'rainy_days']

train_data = train_data[columns_to_keep]
target_data = target_data[columns_to_keep]

# Apply Variance Threshold
climate_features = train_data.drop(columns=['casos', 'date'])
selector = VarianceThreshold(threshold=0.1)
selected_features = selector.fit_transform(climate_features)

# Get the selected feature names
selected_feature_names = climate_features.columns[selector.get_support()]
selected_feature_names = list(selected_feature_names)
print("Selected Features:", selected_feature_names)

# Add 'casos' and 'date' to the selected features
selected_feature_names = ['casos', 'date'] + selected_feature_names

# Update train_data and target_data with selected features
train_data = train_data[selected_feature_names]
target_data = target_data[selected_feature_names]

# Standardize the climate features
scaler = StandardScaler()
train_data[selected_feature_names[2:]] = scaler.fit_transform(train_data[selected_feature_names[2:]])
target_data[selected_feature_names[2:]] = scaler.transform(target_data[selected_feature_names[2:]])

# Prepare the data for Prophet
train_data.rename(columns={'date': 'ds', 'casos': 'y'}, inplace=True)
target_data.rename(columns={'date': 'ds', 'casos': 'y'}, inplace=True)

# Initialize the Prophet model
model = Prophet()

# Add climate features as regressors
for feature in selected_feature_names[2:]:
    model.add_regressor(feature)

# Fit the model
model.fit(train_data)

# Make future DataFrame
future = model.make_future_dataframe(periods=len(target_data), freq='W')

# Add the climate features to the future DataFrame
future = future.merge(climate_data[['date'] + selected_feature_names[2:]], left_on='ds', right_on='date', how='left').drop(columns=['date'])

# Ensure the climate features in the future DataFrame are standardized
future[selected_feature_names[2:]] = scaler.transform(future[selected_feature_names[2:]])

# Make predictions
forecast = model.predict(future)

# Extract the forecasted values for the target period
y_pred_inv = forecast.loc[forecast['ds'].isin(target_data['ds']), 'yhat'].values

# Ensure predicted values are non-negative integers
y_pred_inv[y_pred_inv < 0] = 0
y_pred_inv = y_pred_inv.astype(int)

# Get the actual values
y_target_inv = target_data['y'].values

# Print or return the real cases
print("Actual Cases:", y_target_inv.tolist())
print("Predicted Cases:", y_pred_inv.tolist())

# Calculate MAE and MAPE
mae = mean_absolute_error(y_target_inv, y_pred_inv)
mape = mean_absolute_percentage_error(y_target_inv, y_pred_inv) * 100

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")

# Save the results to a CSV file
results_df = pd.DataFrame({
    'date': target_data['ds'],
    'real': y_target_inv,
    'predict': y_pred_inv
})

results_df.to_csv('ce_task1_prophet_results_with_variance_threshold.csv', index=False)
print("Results saved to 'ce_task1_prophet_results_with_variance_threshold.csv'")


<ipython-input-9-4ca4ad7767c6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengue_train['date'] = pd.to_datetime(dengue_train['date'])
<ipython-input-9-4ca4ad7767c6>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengue_target['date'] = pd.to_datetime(dengue_target['date'])
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input temp

Selected Features: ['temp_min', 'temp_med', 'temp_max', 'precip_max', 'precip_tot', 'rel_humid_min', 'rel_humid_med', 'rel_humid_max', 'thermal_range', 'rainy_days']


DEBUG:cmdstanpy:input tempfile: /tmp/tmpovx4b3gt/1wv2cioq.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=48182', 'data', 'file=/tmp/tmpovx4b3gt/oymkmosf.json', 'init=/tmp/tmpovx4b3gt/1wv2cioq.json', 'output', 'file=/tmp/tmpovx4b3gt/prophet_modeltmid5ag_/prophet_model-20240630174817.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:48:17 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:48:17 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Actual Cases: [216, 186, 132, 133, 165, 161, 143, 142, 106, 74, 47, 65, 163, 127, 170, 226, 240, 246, 274, 354, 482, 577, 561, 614, 581, 444, 543, 509, 476, 636, 689, 666, 615, 512, 475, 514, 424, 362, 340, 339, 291, 253, 272, 216, 210, 224, 188, 134, 133, 104, 88]
Predicted Cases: [0, 0, 0, 0, 161, 98, 123, 115, 186, 181, 0, 47, 79, 0, 240, 420, 460, 773, 651, 829, 961, 902, 1034, 1045, 1683, 1983, 2141, 2188, 2150, 2202, 2313, 2119, 1896, 1745, 1409, 1206, 929, 624, 629, 880, 544, 391, 440, 263, 11, 72, 161, 0, 0, 0, 0]
Mean Absolute Error (MAE): 475.0980392156863
Mean Absolute Percentage Error (MAPE): 121.94909027802944%
Results saved to 'ce_task1_prophet_results_with_variance_threshold.csv'


In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from prophet import Prophet
import pickle

# Load data
dengue_file_path = 'aggregated_dengue_data_ce.csv'
climate_file_path = 'aggregated_climate_data_ce.csv'
dengue_data = pd.read_csv(dengue_file_path)
climate_data = pd.read_csv(climate_file_path)

# Filter for train_1 == True and target_1 == True
dengue_data['train_1'] = dengue_data['train_1'].astype(bool)
dengue_data['target_1'] = dengue_data['target_1'].astype(bool)

dengue_train = dengue_data[dengue_data['train_1']]
dengue_target = dengue_data[dengue_data['target_1']]

# Convert date columns to datetime
dengue_train['date'] = pd.to_datetime(dengue_train['date'])
dengue_target['date'] = pd.to_datetime(dengue_target['date'])
climate_data['date'] = pd.to_datetime(climate_data['date'])

# Merge the datasets on 'date'
train_data = pd.merge(dengue_train, climate_data, on='date')
target_data = pd.merge(dengue_target, climate_data, on='date')

# Select relevant columns excluding 'epiweek'
columns_to_keep = ['date', 'casos', 'temp_min', 'temp_med', 'temp_max',
                   'precip_min', 'precip_med', 'precip_max', 'precip_tot',
                   'pressure_min', 'pressure_med', 'pressure_max',
                   'rel_humid_min', 'rel_humid_med', 'rel_humid_max',
                   'thermal_range', 'rainy_days']

train_data = train_data[columns_to_keep]
target_data = target_data[columns_to_keep]

# Apply Variance Threshold
climate_features = train_data.drop(columns=['casos', 'date'])
selector = VarianceThreshold(threshold=0.1)
selected_features = selector.fit_transform(climate_features)

# Get the selected feature names
selected_feature_names = climate_features.columns[selector.get_support()]
selected_feature_names = list(selected_feature_names)
print("Selected Features:", selected_feature_names)

# Add 'casos' and 'date' to the selected features
selected_feature_names = ['casos', 'date'] + selected_feature_names

# Update train_data and target_data with selected features
train_data = train_data[selected_feature_names]
target_data = target_data[selected_feature_names]

# Standardize the climate features
scaler = StandardScaler()
train_data[selected_feature_names[2:]] = scaler.fit_transform(train_data[selected_feature_names[2:]])
target_data[selected_feature_names[2:]] = scaler.transform(target_data[selected_feature_names[2:]])

# Ensure no NaN values in the climate features before applying PCA
train_data.dropna(inplace=True)
target_data.dropna(inplace=True)

# Apply PCA to climate features
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(train_data[selected_feature_names[2:]])
X_target_pca = pca.transform(target_data[selected_feature_names[2:]])

# Create DataFrames with PCA components
n_components = X_train_pca.shape[1]
train_pca_df = pd.DataFrame(X_train_pca, index=train_data.index, columns=[f'PC{i+1}' for i in range(n_components)])
target_pca_df = pd.DataFrame(X_target_pca, index=target_data.index, columns=[f'PC{i+1}' for i in range(n_components)])

# Add the scaled target variable 'casos_scaled' and 'date' to the PCA DataFrames
train_pca_df['casos'] = train_data['casos'].values
train_pca_df['date'] = train_data['date'].values
target_pca_df['casos'] = target_data['casos'].values
target_pca_df['date'] = target_data['date'].values

# Ensure no NaN values after PCA
train_pca_df.dropna(inplace=True)
target_pca_df.dropna(inplace=True)

# Prepare the data for Prophet
train_pca_df.rename(columns={'date': 'ds', 'casos': 'y'}, inplace=True)
target_pca_df.rename(columns={'date': 'ds', 'casos': 'y'}, inplace=True)

# Initialize the Prophet model
model = Prophet()

# Add PCA components as regressors
for feature in train_pca_df.columns:
    if feature not in ['ds', 'y']:
        model.add_regressor(feature)

# Fit the model
model.fit(train_pca_df)

# Save the model
with open('prophet_model_ce_task1_with_pca.pkl', 'wb') as f:
    pickle.dump(model, f)

# Make future DataFrame
future = model.make_future_dataframe(periods=len(target_pca_df), freq='W')

# Add PCA components to the future DataFrame
future = future.merge(target_pca_df.drop(columns=['y']), on='ds', how='left')

# Ensure no NaN values in the future DataFrame
future.dropna(inplace=True)

# Make predictions
forecast = model.predict(future)

# Extract the forecasted values for the target period
y_pred_inv = forecast.loc[forecast['ds'].isin(target_pca_df['ds']), 'yhat'].values

# Ensure predicted values are non-negative integers
y_pred_inv[y_pred_inv < 0] = 0
y_pred_inv = y_pred_inv.astype(int)

# Get the actual values
y_target_inv = target_pca_df['y'].values

# Print or return the real cases
print("Actual Cases:", y_target_inv.tolist())
print("Predicted Cases:", y_pred_inv.tolist())

# Calculate MAE and MAPE
mae = mean_absolute_error(y_target_inv, y_pred_inv)
mape = mean_absolute_percentage_error(y_target_inv, y_pred_inv) * 100

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")

# Save the results to a CSV file
results_df = pd.DataFrame({
    'date': target_pca_df['ds'],
    'real': y_target_inv,
    'predict': y_pred_inv
})

results_df.to_csv('ce_task1_prophet_results_with_pca.csv', index=False)
print("Results saved to 'ce_task1_prophet_results_with_pca.csv'")


<ipython-input-10-5a3953a74899>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengue_train['date'] = pd.to_datetime(dengue_train['date'])
<ipython-input-10-5a3953a74899>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengue_target['date'] = pd.to_datetime(dengue_target['date'])
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input te

Selected Features: ['temp_min', 'temp_med', 'temp_max', 'precip_max', 'precip_tot', 'rel_humid_min', 'rel_humid_med', 'rel_humid_max', 'thermal_range', 'rainy_days']


17:51:42 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Actual Cases: [216, 186, 132, 133, 165, 161, 143, 142, 106, 74, 47, 65, 163, 127, 170, 226, 240, 246, 274, 354, 482, 577, 561, 614, 581, 444, 543, 509, 476, 636, 689, 666, 615, 512, 475, 514, 424, 362, 340, 339, 291, 253, 272, 216, 210, 224, 188, 134, 133, 104, 88]
Predicted Cases: [0, 0, 0, 0, 30, 38, 0, 0, 120, 108, 0, 0, 60, 41, 87, 394, 340, 532, 538, 662, 934, 873, 849, 1079, 1620, 1804, 2059, 2180, 2171, 2327, 2501, 2171, 1930, 1749, 1433, 1203, 996, 747, 765, 915, 461, 544, 475, 205, 91, 224, 205, 0, 0, 0, 29]
Mean Absolute Error (MAE): 469.72549019607845
Mean Absolute Percentage Error (MAPE): 119.6404358980268%
Results saved to 'ce_task1_prophet_results_with_pca.csv'


In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from prophet import Prophet
import pickle

# List of states and their corresponding file paths
states = ['am', 'ce', 'go', 'mg', 'pr']
tasks = [('train_1', 'target_1'), ('train_2', 'target_2')]

for state in states:
    for train_col, target_col in tasks:
        # Load data
        dengue_file_path = f'aggregated_dengue_data_{state}.csv'
        climate_file_path = f'aggregated_climate_data_{state}.csv'
        dengue_data = pd.read_csv(dengue_file_path)
        climate_data = pd.read_csv(climate_file_path)

        # Filter for train and target columns
        dengue_data[train_col] = dengue_data[train_col].astype(bool)
        dengue_data[target_col] = dengue_data[target_col].astype(bool)

        dengue_train = dengue_data[dengue_data[train_col]]
        dengue_target = dengue_data[dengue_data[target_col]]

        # Convert date columns to datetime
        dengue_train['date'] = pd.to_datetime(dengue_train['date'])
        dengue_target['date'] = pd.to_datetime(dengue_target['date'])
        climate_data['date'] = pd.to_datetime(climate_data['date'])

        # Merge the datasets on 'date'
        train_data = pd.merge(dengue_train, climate_data, on='date')
        target_data = pd.merge(dengue_target, climate_data, on='date')

        # Select relevant columns excluding 'epiweek'
        columns_to_keep = ['date', 'casos', 'temp_min', 'temp_med', 'temp_max',
                           'precip_min', 'precip_med', 'precip_max', 'precip_tot',
                           'pressure_min', 'pressure_med', 'pressure_max',
                           'rel_humid_min', 'rel_humid_med', 'rel_humid_max',
                           'thermal_range', 'rainy_days']

        train_data = train_data[columns_to_keep]
        target_data = target_data[columns_to_keep]

        # Apply Variance Threshold
        climate_features = train_data.drop(columns=['casos', 'date'])
        selector = VarianceThreshold(threshold=0.1)
        selected_features = selector.fit_transform(climate_features)

        # Get the selected feature names
        selected_feature_names = climate_features.columns[selector.get_support()]
        selected_feature_names = list(selected_feature_names)
        print(f"{state.upper()} - Selected Features for {train_col} and {target_col}: {selected_feature_names}")

        # Add 'casos' and 'date' to the selected features
        selected_feature_names = ['casos', 'date'] + selected_feature_names

        # Update train_data and target_data with selected features
        train_data = train_data[selected_feature_names]
        target_data = target_data[selected_feature_names]

        # Standardize the climate features
        scaler = StandardScaler()
        train_data[selected_feature_names[2:]] = scaler.fit_transform(train_data[selected_feature_names[2:]])
        target_data[selected_feature_names[2:]] = scaler.transform(target_data[selected_feature_names[2:]])

        # Ensure no NaN values in the climate features before applying PCA
        train_data.dropna(inplace=True)
        target_data.dropna(inplace=True)

        # Apply PCA to climate features
        pca = PCA(n_components=0.95)
        X_train_pca = pca.fit_transform(train_data[selected_feature_names[2:]])
        X_target_pca = pca.transform(target_data[selected_feature_names[2:]])

        # Create DataFrames with PCA components
        n_components = X_train_pca.shape[1]
        train_pca_df = pd.DataFrame(X_train_pca, index=train_data.index, columns=[f'PC{i+1}' for i in range(n_components)])
        target_pca_df = pd.DataFrame(X_target_pca, index=target_data.index, columns=[f'PC{i+1}' for i in range(n_components)])

        # Add the scaled target variable 'casos' and 'date' to the PCA DataFrames
        train_pca_df['casos'] = train_data['casos'].values
        train_pca_df['date'] = train_data['date'].values
        target_pca_df['casos'] = target_data['casos'].values
        target_pca_df['date'] = target_data['date'].values

        # Ensure no NaN values after PCA
        train_pca_df.dropna(inplace=True)
        target_pca_df.dropna(inplace=True)

        # Prepare the data for Prophet
        train_pca_df.rename(columns={'date': 'ds', 'casos': 'y'}, inplace=True)
        target_pca_df.rename(columns={'date': 'ds', 'casos': 'y'}, inplace=True)

        # Initialize the Prophet model
        model = Prophet()

        # Add PCA components as regressors
        for feature in train_pca_df.columns:
            if feature not in ['ds', 'y']:
                model.add_regressor(feature)

        # Fit the model
        model.fit(train_pca_df)

        # Save the model
        model_filename = f'prophet_model_{state}_{train_col}_{target_col}_with_pca.pkl'
        with open(model_filename, 'wb') as f:
            pickle.dump(model, f)

        # Make future DataFrame
        future = model.make_future_dataframe(periods=len(target_pca_df), freq='W')

        # Add PCA components to the future DataFrame
        future = future.merge(target_pca_df.drop(columns=['y']), on='ds', how='left')

        # Ensure no NaN values in the future DataFrame
        future.dropna(inplace=True)

        # Make predictions
        forecast = model.predict(future)

        # Extract the forecasted values for the target period
        y_pred_inv = forecast.loc[forecast['ds'].isin(target_pca_df['ds']), 'yhat'].values

        # Ensure predicted values are non-negative integers
        y_pred_inv[y_pred_inv < 0] = 0
        y_pred_inv = y_pred_inv.astype(int)

        # Get the actual values
        y_target_inv = target_pca_df['y'].values

        # Print or return the real cases
        print(f"{state.upper()} - Actual Cases for {train_col} and {target_col}: {y_target_inv.tolist()}")
        print(f"{state.upper()} - Predicted Cases for {train_col} and {target_col}: {y_pred_inv.tolist()}")

        # Calculate MAE and MAPE
        mae = mean_absolute_error(y_target_inv, y_pred_inv)
        mape = mean_absolute_percentage_error(y_target_inv, y_pred_inv) * 100

        print(f"{state.upper()} - Mean Absolute Error (MAE) for {train_col} and {target_col}: {mae}")
        print(f"{state.upper()} - Mean Absolute Percentage Error (MAPE) for {train_col} and {target_col}: {mape}%")

        # Save the results to a CSV file
        results_filename = f'{state}_{train_col}_{target_col}_prophet_results_with_pca.csv'
        results_df = pd.DataFrame({
            'date': target_pca_df['ds'],
            'real': y_target_inv,
            'predict': y_pred_inv
        })
        results_df.to_csv(results_filename, index=False)
        print(f"Results saved to '{results_filename}'")


<ipython-input-11-5d2b870abb01>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengue_train['date'] = pd.to_datetime(dengue_train['date'])
<ipython-input-11-5d2b870abb01>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengue_target['date'] = pd.to_datetime(dengue_target['date'])
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


AM - Selected Features for train_1 and target_1: ['temp_min', 'temp_med', 'temp_max', 'precip_max', 'precip_tot', 'rel_humid_min', 'rel_humid_med', 'rel_humid_max', 'thermal_range', 'rainy_days']


DEBUG:cmdstanpy:input tempfile: /tmp/tmpovx4b3gt/fh2j6r8y.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpovx4b3gt/khrl1t4b.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=16560', 'data', 'file=/tmp/tmpovx4b3gt/fh2j6r8y.json', 'init=/tmp/tmpovx4b3gt/khrl1t4b.json', 'output', 'file=/tmp/tmpovx4b3gt/prophet_modelpt9qded1/prophet_model-20240630175404.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:54:04 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:54:05 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


AM - Actual Cases for train_1 and target_1: [250, 196, 161, 147, 129, 117, 118, 110, 87, 52, 44, 81, 198, 195, 198, 214, 251, 277, 180, 148, 167, 143, 125, 119, 122, 125, 106, 153, 135, 143, 127, 131, 89, 65, 75, 69, 78, 90, 103, 115, 93, 89, 57, 46, 75, 65, 67, 77, 75, 73, 57]
AM - Predicted Cases for train_1 and target_1: [0, 0, 0, 131, 15, 0, 26, 80, 85, 54, 57, 50, 159, 260, 394, 441, 491, 550, 600, 688, 631, 563, 512, 474, 274, 277, 180, 98, 79, 16, 43, 0, 0, 0, 0, 155, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
AM - Mean Absolute Error (MAE) for train_1 and target_1: 136.35294117647058
AM - Mean Absolute Percentage Error (MAPE) for train_1 and target_1: 107.42845301692616%
Results saved to 'am_train_1_target_1_prophet_results_with_pca.csv'
AM - Selected Features for train_2 and target_2: ['temp_min', 'temp_med', 'temp_max', 'precip_max', 'precip_tot', 'rel_humid_min', 'rel_humid_med', 'rel_humid_max', 'thermal_range', 'rainy_days']


<ipython-input-11-5d2b870abb01>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengue_train['date'] = pd.to_datetime(dengue_train['date'])
<ipython-input-11-5d2b870abb01>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengue_target['date'] = pd.to_datetime(dengue_target['date'])
<ipython-input-11-5d2b870abb01>:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

...

MG - Actual Cases for train_2 and target_2: [1203, 1547, 1730, 1916, 2909, 3005, 4349, 4991, 5462, 6082, 4753, 5561, 18338, 24030, 37131, 56826, 78217, 91237, 103421, 128992, 133971, 128460, 127278, 127571, 94695, 96086, 85020, 65214, 50304, 41826, 33398, 23354, 14446, 5778, 1314]
MG - Predicted Cases for train_2 and target_2: [2688, 1225, 1829, 2950, 0, 4006, 2900, 1849, 2484, 1171, 1491, 3307, 1581, 4233, 5313, 4962, 5605, 6296, 7736, 9676, 11441, 12400, 13753, 15211, 14320, 15847, 15465, 14927, 14051, 13609, 12573, 11028, 8019, 4905, 3095]
MG - Mean Absolute Error (MAE) for train_2 and target_2: 39121.97142857143
MG - Mean Absolute Percentage Error (MAPE) for train_2 and target_2: 72.45200873644005%
Results saved to 'mg_train_2_target_2_prophet_results_with_pca.csv'
PR - Selected Features for train_1 and target_1: ['temp_min', 'temp_med', 'temp_max', 'precip_max', 'precip_tot', 'rel_humid_min', 'rel_humid_med', 'rel_humid_max', 'thermal_range', 'rainy_days']


DEBUG:cmdstanpy:input tempfile: /tmp/tmpovx4b3gt/4ooxe_12.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=68630', 'data', 'file=/tmp/tmpovx4b3gt/5cy7d_f1.json', 'init=/tmp/tmpovx4b3gt/4ooxe_12.json', 'output', 'file=/tmp/tmpovx4b3gt/prophet_modelobal_awi/prophet_model-20240630175409.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:54:09 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:54:09 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
<ipython-input-11-5d2b870abb01>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

PR - Actual Cases for train_1 and target_1: [266, 366, 422, 439, 563, 680, 777, 791, 933, 1096, 939, 553, 440, 457, 574, 742, 1050, 1588, 2332, 2645, 4357, 6350, 8200, 12578, 15598, 16812, 20908, 19158, 15635, 16117, 15109, 11907, 7831, 5781, 4243, 2870, 1759, 1499, 1220, 878, 643, 579, 555, 508, 489, 488, 378, 354, 459, 488, 503]
PR - Predicted Cases for train_1 and target_1: [1312, 1733, 1831, 1902, 1741, 1956, 1821, 1837, 2074, 2103, 1789, 1885, 2385, 2464, 2972, 3526, 4140, 4247, 5238, 5264, 5686, 5935, 6013, 6642, 6279, 6524, 6211, 6390, 5570, 5097, 4614, 4395, 2955, 2338, 2519, 2007, 2183, 2095, 1684, 1538, 1346, 1252, 2062, 1262, 1579, 1354, 1548, 1576, 1956, 1336, 1220]
PR - Mean Absolute Error (MAE) for train_1 and target_1: 3033.235294117647
PR - Mean Absolute Percentage Error (MAPE) for train_1 and target_1: 163.70994386980908%
Results saved to 'pr_train_1_target_1_prophet_results_with_pca.csv'
PR - Selected Features for train_2 and target_2: ['temp_min', 'temp_med', 'temp_m

DEBUG:cmdstanpy:input tempfile: /tmp/tmpovx4b3gt/sm2rijis.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=41300', 'data', 'file=/tmp/tmpovx4b3gt/fj0kek04.json', 'init=/tmp/tmpovx4b3gt/sm2rijis.json', 'output', 'file=/tmp/tmpovx4b3gt/prophet_model6d3x_kvn/prophet_model-20240630175410.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:54:10 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:54:10 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


PR - Actual Cases for train_2 and target_2: [529, 665, 586, 676, 1027, 1078, 1397, 1473, 2042, 3172, 2582, 2699, 5545, 7182, 9889, 12103, 12548, 15697, 21029, 25552, 27678, 33914, 39538, 44870, 42881, 46870, 45503, 44865, 35280, 36298, 36830, 30880, 19300, 10021, 3162]
PR - Predicted Cases for train_2 and target_2: [2415, 2054, 2381, 2550, 2658, 2025, 2219, 2545, 2878, 3009, 2534, 2629, 2698, 2932, 3406, 3660, 4282, 4497, 4981, 5704, 6211, 6518, 7247, 7322, 7541, 8016, 7687, 7909, 7233, 6472, 5970, 4765, 4460, 3653, 3320]
PR - Mean Absolute Error (MAE) for train_2 and target_2: 14108.57142857143
PR - Mean Absolute Percentage Error (MAPE) for train_2 and target_2: 92.70668217480778%
Results saved to 'pr_train_2_target_2_prophet_results_with_pca.csv'
